In [1]:
import nltk
import pandas as pd
import re
import tensorflow as tf
import time
import multiprocessing
import io
import gensim
import numpy as np
import matplotlib.pyplot as plt
import keras_metrics as km
import pickle
import keras
from sklearn.model_selection import KFold # import KFold
from sklearn.model_selection import StratifiedKFold
from tensorflow import keras
from keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from keras.layers import Embedding
from keras.layers import Dense, Activation, Embedding, LSTM, Bidirectional, Dropout, GRU
from keras import regularizers
from tensorflow.keras.utils import to_categorical
from keras.models import load_model
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn import metrics
from nltk.tokenize import TweetTokenizer
from collections import defaultdict
from datetime import timedelta
from gensim.models import word2vec
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix, accuracy_score, multilabel_confusion_matrix
from sklearn.feature_extraction.text import CountVectorizer
from keras_preprocessing.sequence import pad_sequences
from varname import nameof
import seaborn as sns
from sklearn.utils.multiclass import unique_labels
from sklearn.metrics import classification_report

from nltk.tokenize import word_tokenize
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
import wget


import glob
import os

from datetime import date

import pymysql

In [ ]:
parameter_geo=pd.read_csv("parameter_geo.csv")
# Menghapus spasi dan menggabungkan kata-kata pada kolom "Propinsi"
parameter_geo['Propinsi'] = parameter_geo['Propinsi'].replace(' ', '', regex=True)
for i in range(len(parameter_geo)):
    today = "2023-06-17"#date.today()
    yesterday = "2023-06-18"#today - timedelta(days = 1)
    tokoh="Ganjar Pranowo"
    lokasi=str(parameter_geo.Propinsi[i])
    radius=str(parameter_geo.radius[i])+"km"
       
    twitter_search = f"{tokoh} near:{lokasi} within:{radius} lang:id until:{yesterday} since:{today}"

    # Tentukan nama file dengan format "<kueri pencarian>_<tanggal saat ini>.json"
    filename = "Ganjar.json"
    USING_TOP_SEARCH = True

    snscrape_params = '--jsonl'
    twitter_search_params = ''

    if USING_TOP_SEARCH:
        twitter_search_params += "--top"

    os.system(f"cmd /c snscrape {snscrape_params} twitter-search {twitter_search_params} \"{twitter_search}\" > {filename}")
    tweets_df = pd.read_json(filename, lines=True)
    if len(tweets_df) != 0:    
        tweets_df_rename=tweets_df.rename(columns={"date": "date", "username": "username", "rawContent": "tweet"})
        tweets_df_fix=tweets_df_rename[["date","username", "tweet"]]
        tweets_df_fix["location"] = lokasi
        tweets_df_fix.to_csv(f"hasil_scraping/{lokasi}_GANJAR.csv", index=False)

C:\Users\iki11\AppData\Local\Temp\ipykernel_16784\2877326681.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tweets_df_fix["location"] = lokasi
C:\Users\iki11\AppData\Local\Temp\ipykernel_16784\2877326681.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tweets_df_fix["location"] = lokasi
C:\Users\iki11\AppData\Local\Temp\ipykernel_16784\2877326681.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instea

In [44]:
df_ganjar= pd.concat(map(pd.read_csv, glob.glob(os.path.join('Data/Hasil Crawling/', "*_GANJAR.csv"))))
df_ganjar= df_ganjar.reset_index(drop=True)
df_ganjar = df_ganjar[["date", "username", "tweet", "location"]]
df_ganjar.to_csv("Data/Hasil Crawling/data_ganjar.csv")

In [45]:
ganjar=pd.read_csv('Data/Hasil Crawling/data_ganjar.csv')
ganjar.tail()

,date,username,tweet,location
1307,2023-01-04 07:52:09+00:00,IndahKalula,Ganjar Pranowo pemimpin cerdas dan berkualitas...,SUMATERAUTARA
1308,2023-01-04 07:49:42+00:00,IndahKalula,Ganjar Pranowo sosok yang tepat untuk pimpin I...,SUMATERAUTARA
1309,2023-01-04 16:26:04+00:00,masbin10108833,sehat selalu buat pak Ganjar Pranowo 🇮🇩 https:...,SUMATERAUTARA
1310,2023-01-04 15:24:16+00:00,TUGIMAN_PRIDE,Ganjar Pranowo berkoordinasi dengan Kementeria...,SUMATERAUTARA
1311,2023-01-04 13:00:01+00:00,VIVAcoid,"Survei Indikator Politik Indonesia, memperliha...",SUMATERAUTARA


In [46]:
len(ganjar)

1312

In [47]:
# inisiasi variabel awal
start = 0
end = 1000

# loop untuk membagi data menjadi bagian-bagian dengan jumlah baris 1000
while end <= len(ganjar):
    df_ganjar = ganjar.iloc[start:end, 0:4]
    #menyimpan tweet. (tipe data series pandas)
    data_content = df_ganjar['tweet']
    # casefolding
    data_casefolding = data_content.str.lower()
    #data_casefolding.head()
    #filtering

    #url
    filtering_url = [re.sub(r'''(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'".,<>?«»“”‘’]))''', " ", tweet) for tweet in data_casefolding]
    #cont
    filtering_cont = [re.sub(r'\(cont\)'," ", tweet)for tweet in filtering_url]
    #punctuatuion
    filtering_punctuation = [re.sub('[!"”#$%&’()*+,-./:;<=>?@[\]^_`{|}~]', ' ', tweet) for tweet in filtering_cont]  #hapus simbol'[!#?,.:";@()-_/\']'
    #  hapus #tagger
    filtering_tagger = [re.sub(r'#([^\s]+)', '', tweet) for tweet in filtering_punctuation]
    #numeric
    filtering_numeric = [re.sub(r'\d+', ' ', tweet) for tweet in filtering_tagger]

    # # filtering RT , @ dan #
    # fungsi_clen_rt = lambda x: re.compile('\#').sub('', re.compile('rt @').sub('@', x, count=1).strip())
    # clean = [fungsi_clen_rt for tweet in filtering_numeric]

    data_filtering = pd.Series(filtering_numeric)

    # #tokenize
    tknzr = TweetTokenizer()
    data_tokenize = [tknzr.tokenize(tweet) for tweet in data_filtering]

    #slang word
    path_dataslang = open("dataset/kamus kata baku-clear (1).csv")
    dataslang = pd.read_csv(path_dataslang, encoding = 'utf-8', header=None, sep=";")

    def replaceSlang(word):
        if word in list(dataslang[0]):
            indexslang = list(dataslang[0]).index(word)
            return dataslang[1][indexslang]
        else:
            return word

    data_formal = []
    for data in data_tokenize:
        data_clean = [replaceSlang(word) for word in data]
        data_formal.append(data_clean)
    len_data_formal = len(data_formal)
    # print(data_formal)
    # len_data_formal
    
    nltk.download('stopwords')
    default_stop_words = nltk.corpus.stopwords.words('indonesian')
    stopwords = set(default_stop_words)

    def removeStopWords(line, stopwords):
        words = []
        for word in line:  
            word=str(word)
            word = word.strip()
            if word not in stopwords and word != "" and word != "&":
                words.append(word)

        return words
    data_notstopword = [removeStopWords(line,stopwords) for line in data_formal]
    
    white_list = ["bali"] #ini perlu/tidak perlu diubah karena dianggap sastrawi sebagai imbuhan i

    factory = StemmerFactory()
    ind_stemmer = factory.create_stemmer()
    def stemmer(line):
        temp = list()
        for word in line:
            if(word not in white_list):
                word = ind_stemmer.stem(word)
            if(len(word)>3):
                temp.append(word)
            return temp

    reviews = [stemmer (line) for line in data_notstopword]
    
    #Pembuatan Kamus kata
    t  = Tokenizer()
    fit_text = reviews
    t.fit_on_texts(fit_text)

    #Pembuatan Id masing-masing kata
    sequences = t.texts_to_sequences(reviews)

    #hapus duplikat kata yang muncul
    list_set_sequence = [list(dict.fromkeys(seq)) for seq in sequences]

    #mencari max length sequence
    def FindMaxLength(lst): 
        maxList = max((x) for x in lst) 
        maxLength = max(len(x) for x in lst ) 
        return maxList, maxLength 
        
    # Driver Code 
    max_seq, max_length_seq = FindMaxLength(list_set_sequence)
    jumlah_index = len(t.word_index) +1

    print('jumlah index : ',jumlah_index,'\n')
    # print('word_index : ',t.word_index,'\n')
    # print('index kalimat asli     : ', sequences,'\n')
    # print('kalimat tanpa duplikat : ',list_set_sequence,'\n')
    # print('panjang max kalimat : ', max_length_seq,'kata','\n')
    # print('kalimat terpanjang setelah dihapus duplikat : ', max_seq,'\n')

    count_word = [len(i) for i in list_set_sequence]
    # print('list panjang kalimat : ', count_word)
    max_len_word = max(count_word)
    # print(max_len_word)
    
    padding= pad_sequences([list(list_set_sequence[i]) for i in range(len(list_set_sequence))], 
                       maxlen= 50, padding='pre')
    
    nama_model='gabungan'
    model = load_model('model/'+str(nama_model)+'/Fold'+str(2)+'.h5')
    result=[]
    for test in padding:
        pred = model.predict(np.expand_dims(test,axis=0)).argmax(axis=1)
        result.append(pred[0])
    
    df_ganjar['Tokoh'] = 'Ganjar Pranowo'
    df_ganjar['Sentiment'] = result
    df_ganjar.loc[df_ganjar['Sentiment'] == 2, 'Sentiment'] = -1
    df_ganjar.head()
    
    df_ganjar['Sentiment'].unique()
    
    def insert_to_list_rekomen_db(rp2):
        # Connect to the database
        connection = pymysql.connect(host='127.0.0.1',
                                    port=3306,
                                    user='root',
                                    # password='f#Ur8J3N',
                                    database='klasifikasi_sentimen')
        # create cursor
        cursor=connection.cursor()
        
        cols = "`,`".join([str(i) for i in rp2.columns.tolist()])
        for i,row in rp2.iterrows():
            sql = "INSERT INTO `hasil_scraping` (`" +cols + "`) VALUES (" + "%s,"*(len(row)-1) + "%s)"
            cursor.execute(sql, tuple(row))

            # the connection is not autocommitted by default, so we must commit to save our changes
            connection.commit()
            
    insert_to_list_rekomen_db(rp2=df_ganjar)

    start += 1000
    end += 1000
        

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\iki11\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


jumlah index :  145 

1/1 [==============================] - 0s 18ms/step


In [13]:
#menyimpan tweet. (tipe data series pandas)
data_content = df_ganjar['tweet']
# casefolding
data_casefolding = data_content.str.lower()
#data_casefolding.head()
#filtering

#url
filtering_url = [re.sub(r'''(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'".,<>?«»“”‘’]))''', " ", tweet) for tweet in data_casefolding]
#cont
filtering_cont = [re.sub(r'\(cont\)'," ", tweet)for tweet in filtering_url]
#punctuatuion
filtering_punctuation = [re.sub('[!"”#$%&’()*+,-./:;<=>?@[\]^_`{|}~]', ' ', tweet) for tweet in filtering_cont]  #hapus simbol'[!#?,.:";@()-_/\']'
#  hapus #tagger
filtering_tagger = [re.sub(r'#([^\s]+)', '', tweet) for tweet in filtering_punctuation]
#numeric
filtering_numeric = [re.sub(r'\d+', ' ', tweet) for tweet in filtering_tagger]

# # filtering RT , @ dan #
# fungsi_clen_rt = lambda x: re.compile('\#').sub('', re.compile('rt @').sub('@', x, count=1).strip())
# clean = [fungsi_clen_rt for tweet in filtering_numeric]

data_filtering = pd.Series(filtering_numeric)

# #tokenize
tknzr = TweetTokenizer()
data_tokenize = [tknzr.tokenize(tweet) for tweet in data_filtering]

#slang word
path_dataslang = open("dataset/kamus kata baku-clear (1).csv")
dataslang = pd.read_csv(path_dataslang, encoding = 'utf-8', header=None, sep=";")

def replaceSlang(word):
  if word in list(dataslang[0]):
    indexslang = list(dataslang[0]).index(word)
    return dataslang[1][indexslang]
  else:
    return word

data_formal = []
for data in data_tokenize:
  data_clean = [replaceSlang(word) for word in data]
  data_formal.append(data_clean)
len_data_formal = len(data_formal)
# print(data_formal)
# len_data_formal

In [14]:
nltk.download('stopwords')
default_stop_words = nltk.corpus.stopwords.words('indonesian')
stopwords = set(default_stop_words)

def removeStopWords(line, stopwords):
  words = []
  for word in line:  
    word=str(word)
    word = word.strip()
    if word not in stopwords and word != "" and word != "&":
      words.append(word)

  return words
data_notstopword = [removeStopWords(line,stopwords) for line in data_formal]

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\iki11\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [15]:
white_list = ["bali"] #ini perlu/tidak perlu diubah karena dianggap sastrawi sebagai imbuhan i

factory = StemmerFactory()
ind_stemmer = factory.create_stemmer()
def stemmer(line):
    temp = list()
    for word in line:
      if(word not in white_list):
        word = ind_stemmer.stem(word)
      if(len(word)>3):
        temp.append(word)
    return temp

reviews = [stemmer (line) for line in data_notstopword]
# print(reviews)

In [17]:
#Pembuatan Kamus kata
t  = Tokenizer()
fit_text = reviews
t.fit_on_texts(fit_text)

#Pembuatan Id masing-masing kata
sequences = t.texts_to_sequences(reviews)

#hapus duplikat kata yang muncul
list_set_sequence = [list(dict.fromkeys(seq)) for seq in sequences]

#mencari max length sequence
def FindMaxLength(lst): 
    maxList = max((x) for x in lst) 
    maxLength = max(len(x) for x in lst ) 
    return maxList, maxLength 
      
# Driver Code 
max_seq, max_length_seq = FindMaxLength(list_set_sequence)
jumlah_index = len(t.word_index) +1

print('jumlah index : ',jumlah_index,'\n')
# print('word_index : ',t.word_index,'\n')
# print('index kalimat asli     : ', sequences,'\n')
# print('kalimat tanpa duplikat : ',list_set_sequence,'\n')
# print('panjang max kalimat : ', max_length_seq,'kata','\n')
# print('kalimat terpanjang setelah dihapus duplikat : ', max_seq,'\n')

count_word = [len(i) for i in list_set_sequence]
# print('list panjang kalimat : ', count_word)
max_len_word = max(count_word)
# print(max_len_word)

jumlah index :  2132 



In [18]:
padding= pad_sequences([list(list_set_sequence[i]) for i in range(len(list_set_sequence))], 
                       maxlen= 50, padding='pre')

In [19]:
nama_model='gabungan'
model = load_model('model/'+str(nama_model)+'/Fold'+str(2)+'.h5')
result=[]
for test in padding:
  pred = model.predict(np.expand_dims(test,axis=0)).argmax(axis=1)
  result.append(pred[0])

1/1 [==============================] - 0s 50ms/step


In [20]:
df_ganjar['Tokoh'] = 'Ganjar Pranowo'
df_ganjar['Sentiment'] = result
df_ganjar.loc[df_ganjar['Sentiment'] == 2, 'Sentiment'] = -1
df_ganjar.head()

,date,username,tweet,location,Tokoh,Sentiment
0,2023-04-16 06:05:51,merry_gemoy,Ganjar pranowo berbagi kebahagiaan dengan anak...,ACEH,Ganjar Pranowo,-1
1,2023-04-16 06:05:36,merry_gemoy,Ganjar pranowo raih elektabilitas tinggi di Ja...,ACEH,Ganjar Pranowo,-1
2,2023-04-16 05:55:47,ongapican,@ch_chotimah2 @ganjarpranowo Lonteh pura2 ‘ali...,ACEH,Ganjar Pranowo,-1
3,2023-04-16 05:47:14,MOSLEMSOLIDARI1,@ch_chotimah2 @ganjarpranowo https://t.co/dTj...,ACEH,Ganjar Pranowo,1
4,2023-04-16 05:22:49,fanisalesti,Ganjar pranowo emang keren #GanjarBawaKemajua...,ACEH,Ganjar Pranowo,1


In [21]:
df_ganjar['Sentiment'].unique()

array([-1,  1,  0], dtype=int64)

In [41]:
data=pd.read_csv("data.csv")

In [40]:
def insert_to_list_rekomen_db(rp2):
    # Connect to the database
    connection = pymysql.connect(host='127.0.0.1',
                                 port=3306,
                                 user='root',
                                 # password='f#Ur8J3N',
                                 database='klasifikasi_sentimen')
    # create cursor
    cursor=connection.cursor()
    
    cols = "`,`".join([str(i) for i in rp2.columns.tolist()])
    for i,row in rp2.iterrows():
        sql = "INSERT INTO `hasil_scraping` (`" +cols + "`) VALUES (" + "%s,"*(len(row)-1) + "%s)"
        cursor.execute(sql, tuple(row))

        # the connection is not autocommitted by default, so we must commit to save our changes
        connection.commit()

In [42]:
insert_to_list_rekomen_db(data)